In [1]:
#for device number check
!ls -ltrh /dev/video*

crw-rw---- 1 root video 81, 0 Aug 11 09:35 /dev/video0


In [2]:
from jetcam.usb_camera import USBCamera

# for USB Camera
camera = USBCamera(width=224, height=224, capture_device=0) 

camera.running = True
print("camera ready for use")

camera ready for use


In [3]:
import torchvision.transforms as transforms
from dataset import ImageClassificationDataset

TASK = 'hands'

CATEGORIES = ['describe', 'think', 'worry', 'excited']

DATASETS = ['A', 'B']


TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = ImageClassificationDataset('../data/classification/' + TASK + '_' + name, CATEGORIES, TRANSFORMS)
    
print("{} task with {} categories defined".format(TASK, CATEGORIES))

hands task with ['describe', 'think', 'worry', 'excited'] categories defined


In [4]:
# Set up the data directory location
DATA_DIR = '/nvdli-nano/data/classification/'
!mkdir -p {DATA_DIR}

In [5]:
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

# initialize active dataset
dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera 
camera.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image(layout=ipywidgets.Layout(width="90%"))
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets for data collection
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset', layout=ipywidgets.Layout(width="70%"))
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category', layout=ipywidgets.Layout(width="70%"))
count_widget = ipywidgets.IntText(description='count', layout=ipywidgets.Layout(width="70%"))
save_widget = ipywidgets.Button(description='add', layout=ipywidgets.Layout(width="70%"))

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

# save image for category and update counts
def save(c):
    dataset.save_entry(camera.value, category_widget.value)
    count_widget.value = dataset.get_count(category_widget.value)
save_widget.on_click(save)


print("widgets for data collection are created")

widgets for data collection are created


In [ ]:
import torch
import torchvision


device = torch.device('cuda')

# MOBILENETV2
#model = torchvision.models.mobilenet_v2(pretrained=True)
#model.fc = torch.nn.Linear(512, len(dataset.categories))

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, len(dataset.categories))

# RESNET 34
#model = torchvision.models.resnet34(pretrained=True)
#model.fc = torch.nn.Linear(512, len(dataset.categories))
    
model = model.to(device)

model_save_button = ipywidgets.Button(description='save model', layout=ipywidgets.Layout(width="70%"))
model_load_button = ipywidgets.Button(description='load model', layout=ipywidgets.Layout(width="70%"))
model_path_widget = ipywidgets.Text(description='model path', value='/nvdli-nano/data/classification/my_model.pth', layout=ipywidgets.Layout(width="70%"))

def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))
model_load_button.on_click(load_model)
    
def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)
model_save_button.on_click(save_model)

print(model)

print("model is configured")